In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import multiprocessing as mp
from functools import partial
import cv2
from scipy.signal import convolve2d
from typing import List
from optim_correspondance import load_src_and_dest, fix_timing_between_left_and_right, save_each_frame_to_separate_file
from utils.homography import HomographySIFT, HomographyResult
import torch
from utils.constants import NORM_GL, DENORM_GL, U100C2C
from utils.image_stitcher import make_gif_of_sequence
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable

path_to_files = Path('rawData') / 'calib'
src, dest = load_src_and_dest(path_to_files=path_to_files)

In [ ]:
MAX_TIME_DIFF = 0.001  # seconds
src, dest = fix_timing_between_left_and_right(src=src, dest=dest,
                                              max_time_diff=MAX_TIME_DIFF)

In [ ]:
save_each_frame_to_separate_file(src=src, dest=dest, path_to_files=path_to_files)

# Parameters

In [ ]:
distance_between_centers_of_cameras = 0.1  # meters   ?????????????????????????????????????????
distance_from_ground = 50 # meters
focal_length = 9.8 * 1e-3 # meters
size_of_pixel = 17 * 1e-6 # meters
image_width = 336 # pixels
image_height = 256 # pixels

delta_distance_from_ground = 3 # meters

In [ ]:
fov_width_rad = 2 * np.arctan(image_width * size_of_pixel / (2 * focal_length))
fov_width_degree = fov_width_rad * 180 / np.pi
fov_width_meters = 2 * distance_from_ground * np.tan(fov_width_rad / 2)
fov_height_rad = 2 * np.arctan(image_height * size_of_pixel / (2 * focal_length))
fov_height_degree = fov_height_rad * 180 / np.pi
fov_height_meters = 2 * distance_from_ground * np.tan(fov_height_rad / 2)
print(f"FOV width: {fov_width_degree:.1f}deg, {fov_width_meters:.1f}m")
print(f"FOV height: {fov_height_degree:.1f}deg, {fov_height_meters:.1f}m")


In [ ]:
fov_single_pixel_width_meters = 2 * distance_from_ground * size_of_pixel / focal_length
fov_single_pixel_height_meters = 2 * distance_from_ground * size_of_pixel / focal_length
print(f"FOV single pixel width: {fov_single_pixel_width_meters:.2g}m")
print(f"FOV single pixel height: {fov_single_pixel_height_meters:.2g}m")

min_fov_change = 2 * (distance_from_ground-delta_distance_from_ground) * size_of_pixel / focal_length
max_fov_change = 2 * (distance_from_ground+delta_distance_from_ground) * size_of_pixel / focal_length
print(f"Min FOV change: {min_fov_change:.2g}m")
print(f"Max FOV change: {max_fov_change:.2g}m")

In [ ]:
distance_between_cameras_in_pixels = distance_between_centers_of_cameras / fov_single_pixel_width_meters
print(f"Distance between cameras in pixels: {distance_between_cameras_in_pixels:.2g} pixels")

# An example of the data

In [ ]:
# plot the first frame side-by-side
fig, ax = plt.subplots(1, 2, figsize=(15, 5))
ax[0].imshow(src['frames'][0])
ax[1].imshow(dest['frames'][0])
ax[0].set_title('Left camera')
ax[1].set_title('Right camera')
plt.show()

# Try finding the homography using cv2 on a single image

In [ ]:
# Use cv2 to find the homography between the frames

def find_homography(left, right):
    left = cv2.normalize(left, left, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    right = cv2.normalize(right, right, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    orb = cv2.ORB_create()
    kp1, des1 = orb.detectAndCompute(left, None)
    kp2, des2 = orb.detectAndCompute(right, None)
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    return M, mask
find_homography(src['frames'][0].copy(), dest['frames'][0].copy())[0]

# Brute force search using Kornia
Better results than cv2.

In [ ]:
DEVICE = 'cuda'

with torch.inference_mode():
    homography = HomographySIFT().to(DEVICE, dtype=torch.float32)
    x = np.concatenate([src['frames'][None], dest['frames'][None]], axis=0).transpose(1, 0, 2, 3).copy()
    x = NORM_GL(x.astype(float))
    x = torch.from_numpy(x)
    ret_val: List[HomographyResult] = [homography(x=x_.to(DEVICE, dtype=torch.float32), m=None, mask=None, verbose=False).cpu() for x_ in tqdm(torch.split(x, 1, dim=0))]
h_list = np.concatenate([p.homography for p in ret_val], 0)

### Plot the Kornia results

In [ ]:
# for i, j in np.ndindex(3, 3):
#     d = h_list[:, i, j]
#     std = np.std(d)
#     d = d[np.abs(d - np.mean(d)) < 1 * std]
#     if len(d) == 0:
#         print(f'({i}, {j}) has no data')
#         continue
#     plt.figure()
#     plt.scatter(range(len(d)), d, label='y', s=2)
#     plt.plot(np.mean(d) * np.ones_like(d), label='mean', c='r', linewidth=1)
#     plt.title(f'({i}, {j}), std={d.std():.3g}, mean={np.mean(d):.2g}')
#     plt.grid()
#     plt.show()
#     plt.close()

In [ ]:
indices = np.arange(len(h_list))
np.random.shuffle(indices)
indices = indices[:10]
for idx in indices:
    src_ = src['frames'][idx]
    dest_ = dest['frames'][idx]
    h = ret_val[idx].homography.numpy().squeeze()
    warped = cv2.warpPerspective(src_, h, (src_.shape[1], src_.shape[0])).astype(float)
    mask = cv2.warpPerspective(np.ones_like(src_), h, (src_.shape[1], src_.shape[0])).astype(bool)
    kernel_size = 5
    mask = (convolve2d(mask.astype(float), np.ones((kernel_size, kernel_size)), mode='same') // kernel_size ** 2).astype(bool)
    warped[~mask] = np.nan

    # plot the first frame side-by-side
    fig, ax = plt.subplots(1, 4, figsize=(20, 5))
    ax[0].imshow(src_)
    ax[0].set_title(f'src {np.ptp(src_)=:.0f}')
    ax[0].axhline(src_.shape[0] // 2, c='r', linewidth=1)
    ax[1].imshow(dest_)
    ax[1].set_title(f'dest {np.ptp(dest_)=:.0f}')
    ax[1].axhline(dest_.shape[0] // 2, c='r', linewidth=1)
    ax[2].imshow(warped, vmin=warped[mask].min(), vmax=warped[mask].max())
    ax[2].set_title(f'Warped {np.ptp(warped[mask])=:.0f}')
    ax[2].axhline(warped.shape[0] // 2, c='r', linewidth=1)

    # Calculate the difference between the static and the warped image
    # dest_ = dest_.astype(float)
    # warped = warped.astype(float)
    # dest_ = (dest_ - dest_.min()) / (dest_.max() - dest_.min())
    # warped = (warped - np.nanmin(warped)) / (np.nanmax(warped) - np.nanmin(warped))
    diff = np.abs(dest_ - warped)

    # Add colorbar to ax[3]
    divider = make_axes_locatable(ax[3])
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(ax[3].imshow(diff), cax=cax)
    ax[3].set_title(f'Diff, avg err: {np.nanmean(diff):4.0f}GL')

    plt.show()
    plt.close()